In [1]:
import os

In [2]:
%pwd

'c:\\Sneha\\Kidney-Disease-Classification-Deep-Learning-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Sneha\\Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [34]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILES_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:

        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [35]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import uuid 

In [44]:
class PrepareBaseModel:
    def __init__(self,config: PrepareBaseModelConfig):
        self.config = config
        self.model = None
    

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=(*self.config.params_image_size,3),
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(base_model, classes, freeze_all, freeze_till, learning_rate):
        
    
        # Generate a unique prefix for layer names
        unique_prefix = str(uuid.uuid4())[:8]  
        
        if freeze_all:
            for layer in base_model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in base_model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten(name=f"custom_flatten_{unique_prefix}")(base_model.output)
        prediction = tf.keras.layers.Dense(
            units=classes,
            activation='softmax',
            name=f"custom_dense_{unique_prefix}"
        )(flatten_in)

        full_model = tf.keras.models.Model(
            inputs=base_model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
        

    def update_base_model(self):
        if self.model is None:
            raise ValueError("Base model not initialized. Call get_base_model() first.")

        self.full_model = self._prepare_full_model(
            base_model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)






    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    

In [45]:
try: 
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2025-04-04 07:22:28,860: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-04 07:22:28,864: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-04 07:22:28,866: INFO: common: created directory at: artifacts]
[2025-04-04 07:22:28,868: INFO: common: created directory at: artifacts/prepare_base_model]
[2025-04-04 07:22:31,811: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 